In [51]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
from datetime import datetime, timedelta
import time

# Cấu hình
RSS_URL = "https://rss.app/feeds/z8wTWbZjWSitexso.xml"
START_DATE = "2025-12-01"  # Ngày bắt đầu (lấy từ tháng 12 để có nhiều dữ liệu hơn)
MAX_ITEMS = 100  # Số lượng bài viết tối đa muốn lấy
# END_DATE sẽ tự động = ngày hiện tại (ngày mới nhất)

def fetch_rss_feed(url, max_items=100):
    """Lấy nội dung RSS feed với tham số để lấy nhiều bài hơn"""
    try:
        # Thử thêm tham số limit vào URL
        separator = '&' if '?' in url else '?'
        url_with_params = f"{url}{separator}limit={max_items}"
        
        print(f"🔗 Đang thử lấy tối đa {max_items} bài viết...")
        response = requests.get(url_with_params, timeout=30)
        response.raise_for_status()
        return response.content
    except Exception as e:
        # Nếu thất bại, thử không có tham số
        print(f"⚠️ Thử lại không có tham số limit...")
        try:
            response = requests.get(url, timeout=30)
            response.raise_for_status()
            return response.content
        except Exception as e2:
            print(f"❌ Lỗi khi lấy RSS feed: {e2}")
            return None

def parse_rss_with_date_filter(xml_content, start_date_str):
    """Parse RSS feed với lọc theo ngày - từ ngày chỉ định đến ngày mới nhất"""
    try:
        root = ET.fromstring(xml_content)
        articles = []
        
        # Parse start_date vào đầu ngày (00:00:00)
        start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
        # end_date = ngày hiện tại cuối ngày (23:59:59)
        end_date = datetime.now().replace(hour=23, minute=59, second=59, microsecond=999999)
        
        # Đếm tổng số item trong RSS
        total_items = len(root.findall(".//item"))
        print(f"📦 Tổng số bài trong RSS feed: {total_items}")
        
        for item in root.findall(".//item"):
            title_elem = item.find("title")
            link_elem = item.find("link")
            pub_date_elem = item.find("pubDate")
            description_elem = item.find("description")
            
            if title_elem is not None and link_elem is not None and pub_date_elem is not None:
                title = title_elem.text
                link = link_elem.text
                pub_date_str = pub_date_elem.text
                content = description_elem.text if description_elem is not None else ""
                
                # Parse ngày - hỗ trợ cả 2 định dạng
                try:
                    # Thử định dạng với GMT trước
                    if "GMT" in pub_date_str:
                        pub_date = datetime.strptime(pub_date_str, "%a, %d %b %Y %H:%M:%S GMT")
                    else:
                        pub_date = datetime.strptime(pub_date_str, "%a, %d %b %Y %H:%M:%S %z")
                        pub_date = pub_date.replace(tzinfo=None)
                    
                    # Lọc theo khoảng ngày
                    if start_date <= pub_date <= end_date:
                        articles.append({
                            "title": title,
                            "date": pub_date.strftime("%Y-%m-%d %H:%M:%S"),
                            "link": link,
                            "content": content
                        })
                except Exception as e:
                    print(f"⚠️ Lỗi parse ngày: {pub_date_str} - {e}")
        
        return articles, start_date, end_date, total_items
    except Exception as e:
        print(f"❌ Lỗi khi parse RSS: {e}")
        return [], None, None, 0

def remove_duplicates(articles):
    """Loại bỏ các bài viết trùng link"""
    seen_links = set()
    unique_articles = []
    
    for article in articles:
        if article["link"] not in seen_links:
            seen_links.add(article["link"])
            unique_articles.append(article)
    
    return unique_articles

# Bước 1: Lấy RSS feed
print("🔄 Đang lấy RSS feed...")
xml_content = fetch_rss_feed(RSS_URL, MAX_ITEMS)

if xml_content:
    # Bước 2: Parse và lọc theo ngày
    print(f"🔄 Đang parse RSS feed từ ngày {START_DATE} đến ngày mới nhất...")
    articles, start_date, end_date, total_items = parse_rss_with_date_filter(xml_content, START_DATE)
    
    if end_date:
        print(f"✅ Đã lấy {len(articles)} bài viết trong khoảng ngày (từ {total_items} bài trong feed)")
        print(f"📅 Khoảng thời gian: {start_date.strftime('%Y-%m-%d')} → {end_date.strftime('%Y-%m-%d %H:%M:%S')}")
    
    if len(articles) > 0:
        # Bước 3: Loại bỏ trùng lặp
        unique_articles = remove_duplicates(articles)
        print(f"✅ Sau khi loại bỏ trùng lặp: {len(unique_articles)} bài viết")
        
        # Bước 4: Tạo DataFrame và sắp xếp
        df = pd.DataFrame(unique_articles)
        df['date'] = pd.to_datetime(df['date'])
        df = df.sort_values('date', ascending=False).reset_index(drop=True)
        
        # Hiển thị phân bố theo ngày
        print(f"\n📊 Phân bố theo ngày:")
        date_counts = df['date'].dt.date.value_counts().sort_index(ascending=False)
        for date, count in date_counts.items():
            print(f"  {date}: {count} bài viết")
        
        # Bước 5: Lưu file
        start_str = START_DATE.replace("-", "")
        end_str = end_date.strftime("%Y%m%d")
        output_file = f"theanh28_final_{start_str}_to_{end_str}.csv"
        df.to_csv(output_file, index=False, encoding='utf-8-sig')
        
        file_size = len(df.to_csv(index=False, encoding='utf-8-sig').encode('utf-8-sig')) / 1024
        
        print("\n" + "="*60)
        print("✅ HOÀN THÀNH CRAWL DỮ LIỆU")
        print("="*60)
        print(f"📁 File đã lưu: {output_file}")
        print(f"📊 Tổng số bài viết: {len(df)}")
        print(f"💾 Kích thước file: {file_size:.2f} KB")
        print(f"📅 Từ ngày: {START_DATE}")
        print(f"📅 Đến ngày: {end_date.strftime('%Y-%m-%d')}")
        
        if total_items == len(unique_articles):
            print(f"✅ Đã lấy TẤT CẢ bài viết trong RSS feed")
        else:
            print(f"⚠️ RSS feed có {total_items} bài, nhưng chỉ {len(unique_articles)} bài từ {START_DATE}")
        print("="*60)
        
        # Hiển thị DataFrame
        display(df)
    else:
        print("⚠️ Không có bài viết nào trong khoảng thời gian này")
        print(f"💡 Gợi ý: RSS feed có {total_items} bài, thử đổi START_DATE về trước hơn")
else:
    print("❌ Không thể lấy dữ liệu RSS feed")

🔄 Đang lấy RSS feed...
🔗 Đang thử lấy tối đa 100 bài viết...
🔄 Đang parse RSS feed từ ngày 2025-12-01 đến ngày mới nhất...
📦 Tổng số bài trong RSS feed: 25
✅ Đã lấy 25 bài viết trong khoảng ngày (từ 25 bài trong feed)
📅 Khoảng thời gian: 2025-12-01 → 2026-02-03 23:59:59
✅ Sau khi loại bỏ trùng lặp: 25 bài viết

📊 Phân bố theo ngày:
  2026-02-03: 5 bài viết
  2026-01-16: 20 bài viết

✅ HOÀN THÀNH CRAWL DỮ LIỆU
📁 File đã lưu: theanh28_final_20251201_to_20260203.csv
📊 Tổng số bài viết: 25
💾 Kích thước file: 23.80 KB
📅 Từ ngày: 2025-12-01
📅 Đến ngày: 2026-02-03
✅ Đã lấy TẤT CẢ bài viết trong RSS feed


,title,date,link,content
0,Biểu hiện của sự cố chấp 🤣 Cre: Hiền Nguyễn,2026-02-03 12:32:41,https://www.facebook.com/Theanh28/posts/pfbid0...,"<div><img src=""https://scontent-iad3-1.xx.fbcd..."
1,Theanh28 Entertainment Posted,2026-02-03 12:14:08,https://www.facebook.com/Theanh28/posts/pfbid0...,"<div><img src=""https://scontent-iad3-1.xx.fbcd..."
2,Có thể bạn không biết: Việt Nam có 8 tỷ phú US...,2026-02-03 11:47:09,https://www.facebook.com/Theanh28/posts/pfbid0...,"<div><img src=""https://scontent-iad3-1.xx.fbcd..."
3,Nhận định Arsenal vs Chelsea – Đại chiến Londo...,2026-02-03 11:00:00,https://www.facebook.com/Theanh28/posts/pfbid0...,"<div><img src=""https://scontent-iad3-1.xx.fbcd..."
4,"Valentine này, Cleopatra không chỉ có mùi hươn...",2026-02-03 10:40:02,https://www.facebook.com/Theanh28/posts/pfbid0...,"<div><img src=""https://scontent-iad3-1.xx.fbcd..."
5,Theanh28 Entertainment Posted,2026-01-16 06:44:04,https://www.facebook.com/Theanh28/posts/pfbid0...,"<div><img src=""https://scontent-otp1-1.xx.fbcd..."
6,Theanh28 Entertainment Posted,2026-01-16 06:35:14,https://www.facebook.com/Theanh28/posts/pfbid0...,"<div><img src=""https://scontent-otp1-1.xx.fbcd..."
7,Theanh28 Entertainment Posted,2026-01-16 06:00:20,https://www.facebook.com/Theanh28/posts/pfbid0...,"<div><img src=""https://scontent-otp1-1.xx.fbcd..."
8,Theanh28 Entertainment Posted,2026-01-16 05:31:17,https://www.facebook.com/Theanh28/posts/pfbid0...,"<div><img src=""https://scontent-otp1-1.xx.fbcd..."
9,Theanh28 Entertainment Posted,2026-01-16 05:15:07,https://www.facebook.com/Theanh28/posts/pfbid0...,"<div><img src=""https://scontent-otp1-1.xx.fbcd..."
